In [1]:
import pandas as pd
import numpy as np
import gc
import os
import json

### Settings

In [2]:
with open('settings.json', 'r') as j:
    json_data = json.load(j)

project_path = json_data["project_path"]

data_path = os.path.join(project_path, "data")
python_path = os.path.join(project_path, "python")

raw_data_path = os.path.join(data_path, "raw", "with_loc")
output_data_path = os.path.join(data_path, "output")
supporting_dp = os.path.join(data_path, "supporting")
data_chunks = os.path.join(data_path,"data_chunks")


In [3]:
region_borders_fn = "regions.csv"

In [4]:
python_path

'c:\\Users\\Anna\\Desktop\\AI_camp\\YT\\python'

In [5]:
import sys
sys.path.append(python_path)

In [6]:
import utils

#### Regions borders

In [7]:
#lets take may-16 drives and borders
borders = pd.read_csv(os.path.join(supporting_dp,'regions.csv')
                      , sep = ';', header =0)

In [8]:
borders.head()

,region,west,east,south,north
0,1,-74.25559,-74.244478,40.496120,40.504508
1,2,-74.25559,-74.244478,40.504508,40.512896
2,3,-74.25559,-74.244478,40.512896,40.521285
3,4,-74.25559,-74.244478,40.521285,40.529673
4,5,-74.25559,-74.244478,40.529673,40.538061


### Files to Parce

In [9]:
##### Files review
#subfolder_list = []
#for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
#        subfolder_list.extend(dirnames)

#colum_list = []

# for sbf in subfolder_list:
#     print(sbf)
#     file_list = []
#     for (dirpath, dirnames, filenames) in os.walk(os.path.join(raw_data_path, sbf)):
#         file_list.extend(filenames)
#     #break
#     for f in file_list:
#         print(f)
#         f_path = os.path.join(raw_data_path, sbf, f)
#         dt = pd.read_csv(f_path, sep = ',', header = 0, nrows=3)
       
#         tmp = pd.DataFrame({'sbf': [sbf]*len(dt.columns)
#                             ,'f': [f]*len(dt.columns)
#                             ,'cols': list(dt.columns)})
#         colum_list.append(tmp)

# columns_dt = pd.concat(colum_list, ignore_index = True)
# columns_dt.head()

In [10]:
#columns_dt.shape

In [11]:
#columns_dt.loc[columns_dt.cols == ' dropoff_datetime'].head()

In [12]:
#columns_dt.to_csv(os.path.join(output_data_path, "column_data.csv"), sep=",", header=True, index=False)

#### 2009

In [13]:
sbf = '2016'

file_list = []

for (dirpath, dirnames, filenames) in os.walk(os.path.join(raw_data_path, sbf)):
        file_list.extend(filenames)


In [14]:
file_list

['yellow_tripdata_2016-01.csv',
 'yellow_tripdata_2016-02.csv',
 'yellow_tripdata_2016-03.csv',
 'yellow_tripdata_2016-04.csv',
 'yellow_tripdata_2016-05.csv',
 'yellow_tripdata_2016-06.csv']

### We look aonly on squared that in May 2016 has 5 rides in avergage

In [15]:
f = 'yellow_tripdata_2016-05.csv'
f_path = os.path.join(raw_data_path, sbf, f)
dt = pd.read_csv(f_path, sep = ',', header = 0)
dt = utils.target_column_clean_up(dt)
dt_cl = utils.basic_data_cleaning_and_new_features_wk1(dt)
counts_data = utils.get_counts_per_hour_per_region(dt_cl)

In [16]:
print('Number Of Cellc With No Drive:', counts_data.loc[counts_data.Counts == 0].shape[0])
print('Total number of cells:', counts_data.shape[0])
print('% of Cells With no Drive:', (1.0*counts_data.loc[counts_data.Counts == 0].shape[0])/counts_data.shape[0])

Number Of Cellc With No Drive: 1718238
Total number of cells: 1860000
% of Cells With no Drive: 0.923783870967742


In [17]:
desc_per_square = pd.pivot_table(counts_data, values='Counts', index=['Square_id'],
                  columns=None, aggfunc=[np.sum, np.mean])

borders['Avg'] = pd.Series(desc_per_square.as_matrix()[:,1], index=borders.index)
borders['Sum'] = pd.Series(desc_per_square.as_matrix()[:,0], index=borders.index)

popular = borders.loc[borders.Avg > 5]
print(popular.head())

      region       west       east      south      north         Avg       Sum
1074    1075 -74.022246 -74.011135  40.697437  40.705825   66.799731   49699.0
1075    1076 -74.022246 -74.011135  40.705825  40.714213  138.266129  102870.0
1076    1077 -74.022246 -74.011135  40.714213  40.722601   86.802419   64581.0
1124    1125 -74.011135 -74.000023  40.697437  40.705825   66.041667   49135.0
1125    1126 -74.011135 -74.000023  40.705825  40.714213  181.892473  135328.0


C:\Users\Anna\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Users\Anna\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [18]:
print(len(popular.region.tolist()))

102


In [19]:
target_region_id_list = popular.region.tolist()

In [20]:
fn = os.path.join(supporting_dp, "target_region_id_list.csv")
pd.DataFrame({"target_region_id_list": target_region_id_list}).to_csv(fn, header=True, sep=",", index=False)

In [21]:
del(dt)
del(dt_cl)